# Recurrent Neural Networks!

In this part, we will take part in a real R&D process to build a robust and relevant Recurrent Neural Network. Here is the plan of attack:

- We will study the theory and get the Intuition of RNNs.

- We will start by building a simple RNN, our first attempt.

- We will observe the results to identify possible issues and ways of improvement, so that eventually this simple RNN will be well improved in the last section.

- We will learn how to evaluate a RNN model, and more generally a Regression model.


